# Graphs in ML - Project Notebook
###### Valentin Berkes, Simon Lebastard

In this notebook we will create several strongly and weakly connected graphs, test the Exp3G algorithm and assess the influence of a qualitative change in the connectivity graph on the evolution of regret.

In [ ]:
import networkx as nx
import pygraphviz
from networkx.drawing.nx_agraph import write_dot
from networkx.algorithms.approximation import *
from EXP3 import EXP3, EXP3Opt, compute_regret, plot_regret, gaussian_filter, upper_bound
import arms
import numpy as np
import obsGraph
import pdb

obs_dict = {0:"unobservable", 1:"weakly observable", 2:"strongly observable"}

In [ ]:
G = nx.DiGraph()
G.add_node(0, arm=arms.ArmBernoulli(0.5))
G.add_node(1, arm=arms.ArmBernoulli(0.3))
G.add_node(2, arm=arms.ArmBernoulli(0.4))
G.add_node(3, arm=arms.ArmBernoulli(0.7))
G.add_edge(0,0)
G.add_edge(0,1)
G.add_edge(2,2)
G.add_edge(3,3)

G = nx.convert_node_labels_to_integers(G)

In [ ]:
import matplotlib.pyplot as plt
nx.draw(G)  # networkx draw()
plt.draw()
plt.show()

In [ ]:
## Get nice graphs with self-loops in PNG format
# 1) Install pygraphviz
# 2) Run:
write_dot(G,'graph.dot')
# 3) In terminal, run: dot -Tpng graph.dot > graph.png

We will run the Exp3G algorithm 50 times and produce an average to have a smooth regret function. This will allow us to detect quasi-linear components and better identify the asymptotic regret. Note that quasi-linear components can be used on a transformed regret:
- $x \mapsto \sqrt{x}$ would allow us to find the areas where the regret behaves as $\mathcal{O}(\sqrt{x})$
- $x \mapsto x^{\frac{2}{3}}$ would allow us to find the areas where the regret behaves as $\mathcal{O}(x^{\frac{2}{3}})$

In [ ]:
n_itr = 5000
n_sim = 50

# q,losses = EXP3(G, list(G.nodes()), 0.5, 0.05, n_itr, n_sim)
q,losses = EXP3Opt(G, list(G.nodes()), n_itr, n_sim, weak_dom={0})

In [ ]:
regret = compute_regret(losses, G)

Fitting power functions is still experimental, but it will allow us to track changes in regret trends when it fully works.
Note that we could better fit by knowing the independence number $\alpha(G)$ for strongly connected graphs, or the weak domination number $\delta(G)$ for weakly connected graphs. Determining those values is NP-hard, so no scalable method will be available, but there are some algorithms for computing approximations for small graphs. See for instance (Fox & Pach)

In [ ]:
der2,linAreas = plot_regret(G, [regret, upper_bound(G, len(regret))], ['EXP3', 'Upper bound'])#, reg="Pwr2/3", stdev=34)
# der2,linAreas = plot_regret([regret], ['EXP3'])

Second derivative can be plotted to figure out the thresholds to use for fitting linear and power curves

In [ ]:
#plt.plot(range(10,4700), der2[10:4700])
#plt.show()

## Building strongly connected graphs

To generate a class of strongly connected graphs, we use a parametric method that proceeds as follows:
- A fully connected graph $\mathcal{G}$ is created
- $\alpha \in [0,1]$ specifies the rate of self-edges to be removed from $\mathcal{G}$
- $\beta \in [0,1]$ parametrises the rate of peer-edges to be removed, according to the following policy: if we decide to remove peer-edges for a node $i$ then there is a uniform probability distribution over the number of peer-edges to remove.

Even though this method does allow to generate only a given class of graphs, we can later generalize it by introducing a third parameter $p$ that would be the probability distribution to replace the uniform distribution in the case where peer-edges are removed.

Here is an example of a strongly connected graph created through our function:

In [ ]:
alpha1 = 0.5
beta1 = 0.5
n_nodes = 5
H1 = obsGraph.strong_obs_graph(n_nodes, alpha1, beta1)

In [ ]:
nx.draw(H1)  # networkx draw()
plt.draw()
plt.show()
write_dot(H1,'strong1.dot')

In [ ]:
print("This graph is {}".format(obs_dict[obsGraph.observability_type(H1)]))

In [ ]:
obsGraph.strong_nodes(H1)

Function obsGraph.strong_nodes provides the list of nodes in a graph that are strongly observable.
Those nodes are either observed by themselves ('self'-observed), by all other edges ('peer-observed') or by both ('dual'-observed).

Note that in this case, we chose $\alpha + \beta = 1$, resulting in edges removed for all nodes (either self-edge, or one or more peer-edges).

In the second example below, some edges are left dual:

In [ ]:
alpha2 = 0.4
beta2 = 0.3
n_nodes = 8
H2 = obsGraph.strong_obs_graph(n_nodes, alpha2, beta2)
nx.draw(H2)  # networkx draw()
plt.draw()
plt.show()
#write_dot(H2,'strong2.dot')
print("This graph is {}".format(obs_dict[obsGraph.observability_type(H2)]))
obsGraph.strong_nodes(H2)

# Examples
## Strongly observable
### Bandit

In [ ]:
bandit = nx.DiGraph()
bandit.add_node(0, arm=arms.ArmBernoulli(0.5))
bandit.add_node(1, arm=arms.ArmBernoulli(0.49))
bandit.add_node(2, arm=arms.ArmBernoulli(0.4))
bandit.add_node(3, arm=arms.ArmBernoulli(0.7))
bandit.add_node(4, arm=arms.ArmBernoulli(0.1))
bandit.add_edge(0,0)
bandit.add_edge(1,1)
bandit.add_edge(2,2)
bandit.add_edge(3,3)
bandit.add_edge(4,4)
print("This graph is {}".format(obs_dict[obsGraph.observability_type(bandit)]))

In [ ]:
nx.draw(bandit)
plt.draw()
plt.show()
# Note that networkx does not display self edges
# Exporting the dot graph allows to see self-edges
# write_dot(bandit,'Graphs/bandit.dot')

In [ ]:
n_itr = 10000
n_sim = 100
bandit_q, bandit_losses = EXP3Opt(bandit, list(bandit.nodes()), n_itr, n_sim, alpha=bandit.number_of_nodes()-1)
bandit_regret = compute_regret(bandit_losses, bandit)

In [ ]:
plot_regret(bandit, [bandit_regret, upper_bound(bandit, len(bandit_regret), alpha=bandit.number_of_nodes()-1)], ['EXP3', 'Upper bound'])

In [ ]:
bandit_q

independent set np hard
https://networkx.github.io/documentation/networkx-1.10/reference/algorithms.approximation.html?highlight=independent%20set#module-networkx.algorithms.approximation.independent_set

how to compute weak domination number?

regret doit être une esperance: il faut lancer plusieurs fois et faire la moyenne

### Full feedback

In [ ]:
graph_arms = [arms.ArmBernoulli(0.5), arms.ArmBernoulli(0.3), arms.ArmBernoulli(0.4), arms.ArmBernoulli(0.7), arms.ArmBernoulli(0.1)]
full_feedback = obsGraph.strong_obs_graph(5, 0, 0, graph_arms)
print("This graph is {}".format(obs_dict[obsGraph.observability_type(full_feedback)]))

In [ ]:
nx.draw(full_feedback)  # networkx draw()
plt.draw()
plt.show()
write_dot(bandit,'Graphs/full_feedback.dot')

In [ ]:
n_itr = 10000
n_sim = 100
full_feedback_q, full_feedback_losses = EXP3Opt(full_feedback, list(full_feedback.nodes()), n_itr, n_sim, alpha=1)
full_feedback_regret = compute_regret(full_feedback_losses, full_feedback)

In [ ]:
plot_regret(full_feedback, [full_feedback_regret, upper_bound(full_feedback, len(full_feedback_regret), alpha=1)], ['EXP3', 'Upper bound'])

In [ ]:
min_t = min(len(full_feedback_regret), len(bandit_regret))
plot_regret(full_feedback, [full_feedback_regret[:min_t], bandit_regret[:min_t]], ['Full feedback', 'Bandit'])

### Police officer - loopless clique

In [ ]:
graph_arms = [arms.ArmBernoulli(0.5), arms.ArmBernoulli(0.3), arms.ArmBernoulli(0.4), arms.ArmBernoulli(0.7), arms.ArmBernoulli(0.1)]
police = obsGraph.strong_obs_graph(5, 1, 0, graph_arms)
print("This graph is {}".format(obs_dict[obsGraph.observability_type(police)]))

In [ ]:
nx.draw(police)  # networkx draw()
plt.draw()
plt.show()
#write_dot(bandit,'Graphs/loopless_clique.dot')

In [ ]:
n_itr = 10000
n_sim= 100
police_q, police_losses = EXP3Opt(police, list(police.nodes()), n_itr, n_sim, alpha=1)
police_regret = compute_regret(police_losses, police)

In [ ]:
der2police,_ = plot_regret(police, [police_regret, upper_bound(police, len(police_regret), alpha=1)], ['EXP3', 'Upper bound'])

In [ ]:
min_t = min(len(full_feedback_regret), min(len(bandit_regret),len(police_regret)))
plot_regret(police,
            [full_feedback_regret[:min_t], bandit_regret[:min_t], police_regret[:min_t]],
            ['Full feedback', 'Bandit', 'Police'])

## Weakly observable

### Revealing actions

In [ ]:
revealing = nx.DiGraph()
revealing.add_node(0, arm=arms.ArmBernoulli(0.5))
revealing.add_node(1, arm=arms.ArmBernoulli(0.3))
revealing.add_node(2, arm=arms.ArmBernoulli(0.4))
revealing.add_node(3, arm=arms.ArmBernoulli(0.7))
revealing.add_node(4, arm=arms.ArmBernoulli(0.1))
revealing.add_edge(0,0)
revealing.add_edge(0,1)
revealing.add_edge(0,2)
revealing.add_edge(0,3)
revealing.add_edge(0,4)
print("This graph is {}".format(obs_dict[obsGraph.observability_type(revealing)]))

In [ ]:
nx.draw(revealing)  # networkx draw()
plt.draw()
plt.show()
# write_dot(revealing,'Graphs/revealing.dot')

In [ ]:
n_itr = 10000
n_sim = 100
revealing_q, revealing_losses = EXP3Opt(revealing, list(revealing.nodes()), n_itr, n_sim, weak_dom={0})
revealing_regret = compute_regret(revealing_losses, revealing)

In [ ]:
plot_regret(revealing,
            [revealing_regret, upper_bound(revealing, len(revealing_regret), delta=1)],
            ['EXP3', 'Upper bound'])

In [ ]:
min_t = min(min(len(revealing_regret),len(full_feedback_regret)), min(len(bandit_regret),len(police_regret)))

In [ ]:
plot_regret(revealing,
            [full_feedback_regret[:min_t], bandit_regret[:min_t], police_regret[:min_t], revealing_regret[:min_t]],
            ['Full feedback', 'Bandit', 'Police','Revealing'])

### Cycle

In [ ]:
cycle = nx.DiGraph()
cycle.add_node(0, arm=arms.ArmBernoulli(0.5))
cycle.add_node(1, arm=arms.ArmBernoulli(0.3))
cycle.add_node(2, arm=arms.ArmBernoulli(0.4))
cycle.add_node(3, arm=arms.ArmBernoulli(0.7))
cycle.add_node(4, arm=arms.ArmBernoulli(0.1))
cycle.add_edge(0, 1)
cycle.add_edge(1, 2)
cycle.add_edge(2, 3)
cycle.add_edge(3, 4)
cycle.add_edge(4, 0)
print("This graph is {}".format(obs_dict[obsGraph.observability_type(cycle)]))

In [ ]:
nx.draw(cycle)  # networkx draw()
plt.draw()
plt.show()
#write_dot(cycle,'Graphs/cycle.dot')

In [ ]:
n_itr = 10000
n_sim = 100
cycle_q, cycle_losses = EXP3Opt(cycle, list(cycle.nodes()), n_itr, n_sim, weak_dom={0, 1, 2, 3, 4})
cycle_regret = compute_regret(cycle_losses, cycle)

In [ ]:
plot_regret(cycle, [cycle_regret, upper_bound(cycle, len(cycle_regret), delta=5)], ['EXP3', 'Upper bound'])

In [ ]:
min_t = min(len(cycle_regret),
            min(min(len(revealing_regret),len(full_feedback_regret)), min(len(bandit_regret),len(police_regret))))

In [ ]:
plot_regret(revealing, [revealing_regret, bandit_regret], ['Revealing', 'Bandit'])

In [ ]:
# Revealing and cycle
plot_regret(revealing, [revealing_regret[:min_t], cycle_regret[:min_t]], ['Revealing', 'Cycle'])

In [ ]:
plot_regret(revealing,
            [full_feedback_regret[:min_t], bandit_regret[:min_t], police_regret[:min_t], revealing_regret[:min_t], cycle_regret[:min_t]],
            ['Full feedback', 'Bandit', 'Police','Revealing', 'Cycle'])

## Unobservable

In [ ]:
peer_reveal = revealing.copy()
peer_reveal.remove_edge(0,0)

print("This graph is {}".format(obs_dict[obsGraph.observability_type(peer_reveal)]))

n_itr = 20000
n_sim = 30
peer_reveal_q, peer_reveal_losses = EXP3(peer_reveal, list(peer_reveal.nodes()), 1/2, 1/2, n_itr, n_sim)
peer_reveal_regret = compute_regret(peer_reveal_losses, peer_reveal)

In [ ]:
plot_regret(peer_reveal, [peer_reveal_regret, upper_bound(peer_reveal, len(peer_reveal_regret))], ['EXP3', 'Upper bound'])

# Instability

## Strongly to weakly

Here we will build a strongly connected graph, run Exp3G on this graph but break the strong connectivity while the algorithm runs.
Let's start simple with 5 nodes

### 1) Loopless clique (police) graph

Here we will look at the behavior of a perturbed police graph.

In [ ]:
S1 = police.copy()

In [ ]:
nx.draw(S1)  # networkx draw()
plt.draw()
plt.show()

In [ ]:
n_itr = 20000
n_sim = 30

print("This graph is {}".format(obs_dict[obsGraph.observability_type(S1)]))

alphaS1 = S1.number_of_edges()

deltaS1 = S1.number_of_edges()

In [ ]:
qS1,lS1 = EXP3Opt(S1, list(S1.nodes()), n_itr, n_sim, alpha=alphaS1, delta = deltaS1)
regS1 = compute_regret(lS1, S1)
dr2S1,laS1 = plot_regret(S1, [regS1], ['EXP3 on strong graph'])

In [ ]:
perturbations = [(0,1),(0,2),(0,3),(0,4)
                ,(1,0),(1,2),(1,3),(1,4)
                ,(2,0),(2,1),(2,3),(2,4)
                ,(3,0),(3,1),(3,2),(3,4)
                ,(4,0),(4,1),(4,2),(4,3)]

W1_1 = obsGraph.remove_edges(S1, perturbations, 1)
print("This graph is {}".format(obs_dict[obsGraph.observability_type(W1_1)]))

qW1_1,lW1_1 = EXP3Opt(W1_1, list(W1_1.nodes()), n_itr, n_sim, alpha = alphaS1, delta = deltaS1)
regW1_1 = compute_regret(lW1_1, W1_1)
dr2W1_1,laW1_1 = plot_regret(W1_1, [regW1_1], ['EXP3 on weakly connected graph'])

In [ ]:
W1_4 = obsGraph.remove_edges(S1, perturbations, 4)
print("This graph is {}".format(obs_dict[obsGraph.observability_type(W1_4)]))

qW1_4,lW1_4 = EXP3Opt(W1_4, list(W1_4.nodes()), n_itr, n_sim, alpha = alphaS1, delta = deltaS1)
regW1_4 = compute_regret(lW1_4, W1_4)
dr2W1_4,laW1_4 = plot_regret(W1_4, [regW1_4], ['EXP3 on weakly connected graph'])

Above we run the experiment of removing at first one edge from 0, turning the police graph into a weakly observable graph. Second, we remove other edges leaving zero, without further changing the nature of the observability graph. What happens is that because the learning speed only depends on the nature of the learning graph and, for weakly observable graphs, on $\delta$, furher removing edges from the observability graph without making it unobservable or altering the weak domination number does not change the speed of learning.

### 2) Weakly connected graph, impact of $\delta$

Here we will create a weakly connected graph, with inspiration from the "revealing" graph, such that we can then remove edges from this graph ina way that it stays weakly observable while changing its weak domination number:

In [ ]:
perturbations_FF = [(0,1),(0,2),(0,3),(0,4)]

S2 = revealing.copy()
S2.add_edge(1,2)
S2.add_edge(2,3)
S2.add_edge(3,4)
S2.add_edge(4,1)
print("S2 is {}".format(obs_dict[obsGraph.observability_type(S2)]))


nx.draw(S2, with_labels=True)  # networkx draw()
plt.draw()
plt.show()

In [ ]:
alphaS2 = 1
deltaS2 = 1

W2_1 = obsGraph.remove_edges(S2, perturbations_FF, 1)
W2_2 = obsGraph.remove_edges(S2, perturbations_FF, 2)
W2_3 = obsGraph.remove_edges(S2, perturbations_FF, 3)

print("W2_1 is {}".format(obs_dict[obsGraph.observability_type(W2_1)]))
print("W2_2 is {}".format(obs_dict[obsGraph.observability_type(W2_2)]))
print("W2_3 is {}".format(obs_dict[obsGraph.observability_type(W2_3)]))

In [ ]:
n_itr = 10000
n_sim = 30

qW2_1,lW2_1 = EXP3Opt(W2_1, list(W2_1.nodes()), n_itr, n_sim, alpha = alphaS2, delta = 2)
regW2_1 = compute_regret(lW2_1, W2_1)
dr2W2_1,laW2_1 = plot_regret(W2_1, [regW2_1], ['EXP3G on reveal-1'])

In [ ]:
qW2_2,lW2_2 = EXP3Opt(W2_2, list(W2_2.nodes()), n_itr, n_sim, alpha = alphaS2, delta = 3)
regW2_2 = compute_regret(lW2_2, W2_2)
dr2W2_2,laW2_2 = plot_regret(W2_2, [regW2_2], ['EXP3G on reveal-2'])

In [ ]:
qW2_3,lW2_3 = EXP3Opt(W2_3, list(W2_3.nodes()), n_itr, n_sim, alpha = alphaS2, delta = 4)
regW2_3 = compute_regret(lW2_3, W2_3)
dr2W2_3,laW2_3 = plot_regret(W2_3, [regW2_3], ['EXP3G on reveal-3'])

The learning rate depends on $\delta^{\frac{1}{3}}$, which can be observed in our case.

Here we looked at the impact of the weak domination number in the case of weakly connected graphs. However we could have lead similar experiments to follow the impact of the independence number in the case of strongly connected graphs.

## Strongly to unobservable

A good example of transition from strongly observable to unobservable graph is the case of the bandit graph when one edge is removed. This implies that one of the vertices is no longer observed neither by himself nor by any other vertex, which implies that not all vertices are observable.

In [ ]:
alpha1 = 1
beta1 = 0
n_nodes = 5
S3 = bandit.copy()

In [ ]:
nx.draw(S3)  # networkx draw()
plt.draw()
plt.show()

n_itr = 20000
n_sim = 30

print("This graph is {}".format(obs_dict[obsGraph.observability_type(S2)]))

alphaS3 = S3.number_of_nodes()

deltaS3 = S3.number_of_nodes()

In [ ]:
qS3,lS3 = EXP3Opt(S2, list(S2.nodes()), n_itr, n_sim, alpha=alphaS3, delta = deltaS3)
regS3 = compute_regret(lS3, S3)
dr2S3,laS3 = plot_regret(S3, [regS3, upper_bound(S3, len(regS3), alphaS3, deltaS3)], ['EXP3 bandit graph 5 nodes', 'Upper bound'])

In [ ]:
perturbations = [(0,0),(1,1),(2,2),(3,3),(4,4)]

U2_1 = obsGraph.remove_edges(S3, perturbations, 1)
print("This graph is {}".format(obs_dict[obsGraph.observability_type(U2_1)]))

qU2_1,lU2_1 = EXP3Opt(U2_1, list(U2_1.nodes()), n_itr, n_sim, alpha = alphaS3, delta = deltaS3)
regU2_1 = compute_regret(lU2_1, U2_1)
dr2U2,laU2 = plot_regret(U2_1, [regU2_1, upper_bound(U2_1, len(regU2_1), alphaS3, deltaS3)], ['EXP3, unobservable mod. bandit 5 nodes', 'Upper bound'])

In [ ]:
U2_2 = obsGraph.remove_edges(S3, perturbations, 2)
print("This graph is {}".format(obs_dict[obsGraph.observability_type(U2_2)]))

qU2_2,lU2_2 = EXP3Opt(U2_2, list(U2_2.nodes()), n_itr, n_sim, alpha = alphaS3, delta = deltaS3)
regU2_2 = compute_regret(lU2_2, U2_2)
dr2U2,laU2 = plot_regret(U2_2, [regU2_2, upper_bound(U2_2, len(regU2_2), alphaS3, deltaS3)], ['EXP3, unobservable mod. bandit 5 nodes', 'Upper bound'])

In [ ]:
U2_3 = obsGraph.remove_edges(S3, perturbations, 3)
print("This graph is {}".format(obs_dict[obsGraph.observability_type(U2_3)]))

qU2_3,lU2_3 = EXP3Opt(U2_3, list(U2_3.nodes()), n_itr, n_sim, alpha = alphaS3, delta = deltaS3)
regU2_3 = compute_regret(lU2_3, U2_3)
dr2U2,laU2 = plot_regret(U2_3, [regU2_3, upper_bound(U2_3, len(regU2_3), alphaS3, deltaS3)], ['EXP3, unobservable mod. bandit 5 nodes', 'Upper bound'])

## Bibliography

Fox & Pach, Computing the Independence Number of Intersection Graphs, math.mit.edu/~fox/paper-foxj.pdf